In [ ]:
import numpy as np
import sys
import os
import plotly.graph_objects as go
import plotly.express as px
sys.path.append('../Functions/py_functions/') # This path is so that within each function file, you can import the other function files with relative paths
sys.path.append('../') # This path is so that we can import the functions folder from the root directory compared to where this file is
from Functions.py_functions.constants import *
from Functions.py_functions.car_configuration import Car
from Functions.py_functions.gps_importer import *
from Functions.py_functions.mmd import MMD
from Functions.py_functions.tire_model.tire_model_utils import *

In [ ]:
v_avg = 15
max_beta = 35
max_delta = 30
size = 50
use_lin = True
# car = Car()
rsd = 0.25
total_roll_stiffness = 900 * FTLB_TO_NM
car = Car(front_axle_weight=0.4725)
car.z_f = 0.13 * IN_TO_M
car.z_r = 1.0 * IN_TO_M
car.cdA = 1.33
car.clA = 3.43
car.cg_height=10.7*IN_TO_M
car.front_axle_downforce = 0.48
car.k_f = total_roll_stiffness * rsd
car.k_r = total_roll_stiffness * (1-rsd)
car.k_c = 1000000
car.update_car()
car.set_lltd()
car.set_tire(H_R20_18X6_7)
# car.fast_mf = None

In [ ]:
# sweep f_z with the tire model
fzs = np.linspace(0, 3500, 100)
kappas, bams, fxs = [], [], []
kappas_o, bams_o, fxs_o = [], [], []
for fz in fzs:
    s_a, upper, lower, p, i_a, v, mu_corr, flip_s_a, non_driven, fx_targ = -0.1909507348225375, 0.20, -0.30, 82500, -0.0055692567224981705, 14.572984415505347, 0.65, True, False, 750.5571514851151
    kappa_o, bam_o, fx_o = car.s_r(fz, s_a, v, fx_targ, i_a=i_a, non_driven=non_driven, upper=upper, lower=lower, og_lower=lower, og_upper=upper, flip_s_a=flip_s_a, mu_corr=mu_corr)
    kappa, bam, fx = car.fast_mf.s_r(fz, s_a, upper, lower, upper, lower, 0.0, 0.0, 0.0, p, i_a, v, 0.0, 0.0, mu_corr, flip_s_a, non_driven, fx_targ, 0)
    kappas.append(kappa)
    bams.append(bam)
    fxs.append(fx)
    kappas_o.append(kappa_o)
    bams_o.append(bam_o)
    fxs_o.append(fx_o)

px.scatter(x=fzs, y=kappas, labels={'x': 'fz', 'y': 'kappa'}).show()
# px.scatter(x=fzs, y=kappas_o, labels={'x': 'fz', 'y': 'kappa_o'}).show()
px.scatter(x=fzs, y=bams, labels={'x': 'fz', 'y': 'bam'}).show()
# px.scatter(x=fzs, y=bams_o, labels={'x': 'fz', 'y': 'bam_o'}).show()
px.scatter(x=fzs, y=fxs, labels={'x': 'fz', 'y': 'fx'}).show()
# px.scatter(x=fzs, y=fxs_o, labels={'x': 'fz', 'y': 'fx_o'}).show()


In [ ]:
# plot a 2d surface of fx with the axis of s_r and f_z
s_r = np.linspace(-0.3, 0.2, 100)
f_z = np.linspace(0.0, 3500, 200)
s_r_a, f_z_a = np.meshgrid(s_r, f_z)
fx = np.zeros_like(s_r_a)
for i in range(len(s_r)):
    for j in range(len(f_z)):
        fxx, _, _ = car.mf_tire.steady_state_mmd(f_z[j], s_a, s_r[i], v=v_avg, flip_s_a=flip_s_a, i_a=i_a, mu_corr=mu_corr)
        # fxx, _, _ = car.fast_mf.solve_steady_state(f_z[j], s_a, s_r[i], p, i_a, v_avg, 0.0, 0.0, mu_corr, flip_s_a)
        fx[j][i] = fxx

fig = go.Figure(data=[go.Surface(z=fx, x=s_r_a, y=f_z_a)])
fig.show() 

In [ ]:
kappa = 0.2
fz0_prime = car.mf_tire.LFZO * car.mf_tire.FNOMIN
dfz = (f_z - fz0_prime) / fz0_prime
dpi = (p - car.mf_tire.NOMPRES) / car.mf_tire.NOMPRES
Cx = car.mf_tire.PCX1 * car.mf_tire.LCX
Vsx = -kappa * np.abs(v_avg)
Vsy = np.tan(s_a) * np.abs(v_avg)
Vs = np.sqrt(Vsx**2 + Vsy**2)
LMUX_star = car.mf_tire.LMUX / (1 + car.mf_tire.LMUV * Vs / car.mf_tire.LONGVL)
LMUX_prime = LMUX_star
mux = (car.mf_tire.PDX1 + car.mf_tire.PDX2 * dfz) * (1 + car.mf_tire.PPX3 * dpi + car.mf_tire.PPX4 * dpi**2) * (1 - car.mf_tire.PDX3 * i_a**2) * LMUX_star
Dx = mux * f_z
Kxk = f_z * (car.mf_tire.PKX1 + car.mf_tire.PKX2 * dfz) * np.exp(car.mf_tire.PKX3 * dfz) * (1 + car.mf_tire.PPX1 * dpi + car.mf_tire.PPX2 * dpi**2) * car.mf_tire.LKX
def safe_sign(arr):
    if type(arr) != np.ndarray: # Zero Fz correction
        if arr >= 0:
            return 1.0
        return -1.0
    sign = np.sign(arr)
    sign[sign == 0] = 1
    return sign
signDx = safe_sign(Dx)
Bx = Kxk / (Cx * Dx + car.mf_tire.epsilon * signDx)
SHx = (car.mf_tire.PHX1 + car.mf_tire.PHX2 * dfz) * car.mf_tire.LHX  # (4.E17)
SVx = f_z * (car.mf_tire.PVX1 + car.mf_tire.PVX2 * dfz) * car.mf_tire.LVX * LMUX_prime
kappax = kappa + SHx
Ex = (car.mf_tire.PEX1 + car.mf_tire.PEX2 * dfz + car.mf_tire.PEX3 * dfz**2) * (1 - car.mf_tire.PEX4 * np.sign(kappax)) * car.mf_tire.LEX
if type(Ex) == np.ndarray: # Zero Fz correction
    Ex[Ex > 1] = 1
elif Ex > 1:
    Ex = 1
Fx0 = Dx * np.sin(Cx * np.arctan(Bx * kappax - Ex * (Bx * kappax - np.arctan(Bx * kappax)))) + SVx
px.scatter(x=f_z, y=dfz, title="dfz").show()
px.scatter(x=f_z, y=(car.mf_tire.PKX1 + car.mf_tire.PKX2 * dfz), title="PKX1&2").show()
px.scatter(x=f_z, y=np.exp(car.mf_tire.PKX3 * dfz), title="PKX3").show()
px.scatter(x=f_z, y=mux, title="mux").show()
px.scatter(x=f_z, y=Dx, title="Dx").show()
px.scatter(x=f_z, y=Kxk, title="Kxk").show()
px.scatter(x=f_z, y=Bx, title="Bx").show()
px.scatter(x=f_z, y=SHx, title="SHx").show()
px.scatter(x=f_z, y=SVx, title="SVx").show()
px.scatter(x=f_z, y=kappax, title="kappax").show()
px.scatter(x=f_z, y=Ex, title="Ex").show()
px.scatter(x=f_z, y=Fx0, title="Fx0").show()